# Capstone Project - The Battle of the Neighborhoods
## Finding the best location to open a fast food restaurant in Toronto

<a id='intro'></a>
## 1. Introduction
The project is aimed at leveraging the Forsquare location data to solve the problem of identifying the best neighborhood in which to open a new fast food restaurant in Toronto. Details of the project as well as processing of the data used will be described and a neighborhood will be recommended to the client. 

## 1.1 Background Information
One of the many decisions that needs to be made when opening a new business is about the location in which the business should be opened. There are usually a lot of factors that may be considered for a location depending on the specific business. One good pointer as to whether a specific area will be potentially good for a business is to consider if there is a similar business in the same or similar areas to the one being considered. In this project, we will explore how information about different locations can be used to decide where to open a new business. Specifically, we are interested in helping a client find the best location for a new fast food (FF) restaurant in Toronto.

## 1.2 Problem Statement
Our client, Maria is planning to open a new fast food (FF) restaurant in Toronto but needs guidance on the best neighborhood in whihc she can open the restaurant. Mary is not yet decided on the specific fast food (e.g KFC, McDonalds, Burger King) but she is certain that it will be a fast food restaurant. Based on initial conversations with mary, she believes that neighborhoods with an exisiting FF restaurant is a viable option. Neighborhoods that also look similar (in terms of surrounding venues) to another neighborhood with a FF restaurant are also likely to be a good location. However, opening a FF restaurant in a location that already has many FF restaurants or within close proximity to another FF restaurant will create more competition that Maria would prefer to avoid for her new business. Since there are so many neighborhoods, Maria would liek to know whihc neighbourhood is the best location in whichi to open a new FF restaurant in Toronto.

## 1.3 Target Audience
This project is targeted at helping a client - Maria to know where to open a new FF restaurant in Toronto based on how many restaurants are currently in each neighborhood and the similarity of each neighborhood to ones with existing fast food restaurants. A similar approach to this analysis can be employed to find the best location for other businesses as well.

<a id='data'></a>
## 2. Data
This project uses data from different sources to address the problem stated. Information about neighborhoods in Toronto will be extracted from a web page. Detailed information about the venues in each neighborhood will then be gathered from Foursquare for further processing

### 2.1 Description of Data
The following data are used in the project.
1. Borough and neighborhoods information: This information is gathered by webscraping a wikipedia page (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) that contains a list of the postal codes, borough, and neighborhoods of Toronto. It is then stored in a data frame. 
2. Longitude and Latitude information: In order to explore a neighborhood using Foursquare, we need the latitude and longitude of the locations. There are two ways in which this information could be gathered. The first approach is to make use of the geocoder package, which provides the latitude and longitude of a given location/postal code. The alternative is to use a CSV file in which this information had already been stored (https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv). The project uses this alternative method and reads the latitude and longitude of each neighborhood form the CSV file.
3. Venues in each neighborhood: In order to analyze the neighborhoods in Toronto, we need more information about the venues around each neighborhood. Foursquare location services provide a way to gather this information and will be used to get data on venues in each neighborhood of Toronto. 

Although it is possible to limit our analysis to only neighborhoods with 'Toronto' in its name, we decided against this in order to have more neighborhood venues to be explored for the best location for a FF restaurant. 

### 2.2 How the Data will be used
The data will be used to arrive at a recommendation of a neighbourhood where a new FF restaurant is best located. 

We will start out with a data frame containing all the neighbourhoods in Toronto. Latitudes and Longitudes of the neighborhoods will then be added to the data frame. This infromation will be used to query Foursquare location services to obtain venues in each neighborhood. Each venue usually belongs to a venue category in the obtained data. For the purpose of the project, we are particularly interest in the Fast Food Restaurant category since it describes exactly the business that our client is interested in opening. However, to better understand the neighborhoods and how similar they are to each other, the complete set of data for all neighborhoods and venue categories will be used to cluster the neighborhoods into groups of similar neighborhoods.

Using the data frame that contains the complete list of venues in each neighborhood, we do some data cleaning to prepare the data for one-hot ending of the venue categories. We will then use the resulting data of one-hot encoding to do some exploratory analysis to see some of the most common venues in each neighborhood. In order to identify neighborhoods that are similar to one another, the _k_-means model will be used to cluster the neighborhood into a number of clusters. The method employed to identify the best number for _k_ is the Elbow method. 

Throughout the project report, we willl use plots and Folium maps for visual exploratory analysis and understanding of the neighborhoods and other data.

Once we have the clusters defined, we will explore each cluster more closely and find the average number of FF restuarants in each cluster. Our goal is to find the top 2 clusters with the highest average number of restaurants within the cluster. We decided to use the top 2 clusters in order to have more neighborhoods to further explore. Although having highest average number of FF restaurants in a cluster would imply that there is demand for fast food in that cluster, we also want to limit the amount of competition that the business would face when opened in a neighborhood within the cluster. To find the best neighborhood for the business, we will calculate a distance matrix to show the distance between neighborhoods with no existing FF restaurant and those with at least one FF restaurant within the best clusters earlier identified. Distance to neighborhoods with multiple FF restaurant will be multiplied by the number of restaurants in the neighborhood for a better analysis of the competition that may be faced. Finally we find the maximum average distance across all neighborhoods that currently have no FF restaurant but which are within the best clusters identified. The maximum average distance implies that it has the least amount of competition with existing FF restaurants that are in the best clusters earlier identified but it is still within a neighborhood where FF restaurants are in high demand. 

The analysis will lead to the identification of the best 2 neighborhoods in which our client may open a new fast food restaurant in Toronto. The neighborhoods are shown on the map together with other neighborhoods and their clustering. Neighborhoods will existing fast food restaurants will also be identified on the final map for a visual identification of the best neighborhoods found for the new business.

